In [1]:
%cd ../

/Users/復讐者/Dropbox/CourseZero


In [2]:
import json
import pandas as pd

import requests
from bs4 import BeautifulSoup
from CourseZero.FileSystemTools import getSystemRoot


from IPython.display import display
from ipywidgets import widgets

from CourseZero import environment as env

data_json = "{}/chdata.json".format(env.DATA_FOLDER)
output_file = "{}/coursehero-draft.csv".format(env.STORAGE_FOLDER)

In [4]:
from CourseZero.Store import DataStore
from CourseZero.Widgets.CampusAndProfIdentifiers import make_campus_selector

# Load campus ids from file

In [3]:
from CourseZero.FileSystemTools import getSystemRoot
from CourseZero.DataStorageTools import load_campus_id_data

csu_id_data = '{}/csu_ids.json'.format(env.DATA_FOLDER)

campus_ids = load_campus_id_data(csu_id_data)
csu_names = [c['name'] for c in campus_ids]

# Select your campus

In [ ]:
make_campus_selector(DataStore, campus_ids)

# courses

# course search

In [10]:
def parse_json_into_df(json_data):
    """Parse json data into a usable dataframe
    For now: Control which columns are kept by commenting out in to_keep
    """
    to_keep = [
#         '@timestamp', 
     'canonical_course_id', 
#         'country', 
        'course_id',
       'course_info', 
        'course_name', 
        'course_num', 
#         'course_pk', 
        'dept_acro',
       'dept_id', 
        'dept_name', 
        'doc_counts', # if this isn't removed, can't dedupe
#         'location', 
        'prof_id',
       'prof_name', 
#         'school_aliases', 
#         'school_id', 
        'school_name',
#        'subdivision', 
        'term', 
        'title', 
        'total_doc_count', 
        'type', 
        'url',
       'verified', 
        'year'
    ]

    data = []

    for r in json_data:
        try:
            if r['total'] >0: 
                for row in r['results']:
                    data.append(row)

        except Exception as e:
            pass
    
    data = pd.DataFrame(data)
    data = data[to_keep]

    # Parse the counts of each kind of document out
    def parse_counts(row):
        for c in row['doc_counts']:
            row[c['category']] = c['count']
        return row
    data = data.apply(lambda r: parse_counts(r), axis=1)
    # We need to drop the counts column so can de-dupe
    data.drop(['doc_counts'], axis=1, inplace=True)
    data.drop_duplicates(inplace=True)
    return data

# Data frame operations
def get_departments(frame):
    # Extract standard list of departments
    depts = list(set(frame.dept_acro.tolist()))
    depts.sort()
    return depts

def normalize_prof_name(prof_name):
    return prof_name.strip().upper()

# Frame filtration operations
def filter_by_dept_abbrevs(frame):
    return frame[frame['dept_acro'].isin(DataStore.departments)]

def get_by_course_id(frame, course_ids):
    if type(course_ids) is not list:
        course_ids = list(course_ids)
    return frame[frame['course_id'].isin(course_ids)]
#     return frame[frame['course_id'] == course_id]

In [ ]:
len(csus)

In [ ]:
# Url for finding campus ids
from CourseZero.UrlMakers import make_campus_search_url, make_course_search_url

# def make_campus_search_url(campus):
#     campus_search_url = "https://www.coursehero.com/ajax/autocomplete_resultset.php?term=%s&type=school"
#     return campus_search_url % campus
# 
# def make_course_search_url(query, csu_id):
#     course_search_url = "https://www.coursehero.com/api/v1/courses/search/?q=%s&schoolId=%s"
#     return course_search_url % (query, csu_id)

In [ ]:
url = make_campus_search_url('Chico')
r = requests.get(url, headers=headers)
r.json()

# Campus id finding

In [ ]:
campus_ids = [('California State University Bakersfield', '262'),
              ('California State University, Chico', '264'),
 ('California State University Channel islands', '263'),
 ('California State University Dominguez hills', '265'),
 ('California State University East bay', '266'),
 ('California State University Fresno', '267'),
 ('California State University Fullerton', '268'),
 ('Humboldt State University', '3303'),
 ('California State University Long beach', '270'),
 ('California State University Los Angeles', '269'),
 ('California Maritime Academy', '245'),
              ('California State University Monterey Bay', '3348' ),
 ('California State University Northridge', '272'),
 ('California Polytechnic State University, San Luis Obispo', '50'),
 ('California State University San bernardino', '274'),
('California State University, Sacramento', '273'),
 ('San Diego State University', '24'),
 ('San Francisco State University', '362'),
 ('San jose State University', '19'),
 ('Sonoma State University', '382'),
 ('California State University San Marcos', '275'),
 ('California State University Stanislaus', '276'),
('California Polytechnic State University, Pomona', '248'),
]

## campus doc counts

In [ ]:
document_counts = []

for r in results:
#     print(r[0]['document_count'])
    document_counts.append((r[0]['label'], r[0]['document_count']))
    print(r[0]['label'], r[0]['document_count'])
#     print(list(r.decode('utf-8')))

In [5]:
from CourseZero.RequestTools import get_docs_for_campuses, get_docs_for_campus

In [ ]:
json_data = get_docs_for_campus(268, 'csuf')

Searching for csun


26

In [ ]:
json_data

Responses look like this

[{'total': 0, 'results': []},
 {'total': 25,
  'results': [{'course_id': '1423503',
    'canonical_course_id': '1423503',
    'course_num': '280',
    'course_name': 'BUSINESS LAW',
    'dept_id': '43776',
    'dept_acro': 'BLAW',
    'dept_name': '',
    'school_id': '272',
    'school_name': 'California State University, Northridge',
    'prof_id': '275168',
    'prof_name': 'NG',
    'course_pk': '1423503',
    'verified': '0',
    'year': '2011',
    'term': '3',
    'country': 'United States',
    'subdivision': 'California',
    'school_aliases': 'Cal State Northridge California State University Northridge California State University, Northridge CSU Northridge CSUN San Fernando Valley State College',
    'course_info': 'BLAW 280 BUSINESS LAW',
    'url': '/sitemap/schools/272-California-State-University-Northridge/courses/1423503-BLAW280/',
    'total_doc_count': 238,
    'doc_counts': [{'category': 'Essays', 'count': '17'},
     {'category': 'Homework Help', 'count': '23'},
     {'category': 'Notes', 'count': '149'},
     {'category': 'Test Prep', 'count': '13'}],
    'location': {'lat': '34.23550300', 'lon': '-118.52775300'},
    'title': 'California State University, Northridge - BLAW 280 - BUSINESS LAW',
    '@timestamp': '2018-07-27T18:02:22.000Z',
    'type': 'course'},
   {'course_id': '1266538',
    'canonical_course_id': '1266538',
    'course_num': '308',
    'course_name': 'business law 11',
    'dept_id': '43776',
    'dept_acro': 'BLAW',
    'dept_name': '',
    'school_id': '272',
    'school_name': 'California State University, Northridge',
    'prof_id': '232909',
    'prof_name': 'zucker',
    'course_pk': '1266538',
    'verified': '0',
    'year': '2010',
    'term': '3',
    'country': 'United States',
    'subdivision': 'California',
    'school_aliases': 'Cal State Northridge California State University Northridge California State University, Northridge CSU Northridge CSUN San Fernando Valley State College',
    'course_info': 'BLAW 308 business law 11',
    'url': '/sitemap/schools/272-California-State-University-Northridge/courses/1266538-BLAW308/',
    'total_doc_count': 204,

In [8]:
data = parse_json_into_df(json_data)

In [9]:
len(data)

364

In [11]:
depts = get_departments(data)

In [12]:
def make_dept_b(dept):
    b = widgets.Button(description=dept, button_style='primary')
    def handle(event):
        if dept in DataStore.departments:
            DataStore.remove_department(dept)
            b.button_style='primary'
        else:
            DataStore.add_department(dept)
            b.button_style='success'
    b.on_click(handle)
    return b


def split_list(a_list):
    half = len(a_list)//2
    return a_list[:half], a_list[half:]


buttons = []
for dept in depts:
    buttons.append(make_dept_b(dept))

b1, b3 = split_list(buttons)
b1, b2 = split_list(b1)
b3, b4 = split_list(b3)

The site does not seem to standardize the departments students enter. Thus your class may be identified in some rather odd ways (it's almost as though someone wanted it to be hard)...

Run the next cell and select all the possible departments using the buttons which display.

In [13]:
display(widgets.HBox([widgets.VBox(b1), widgets.VBox(b2), widgets.VBox(b3), widgets.VBox(b4)]))

In [14]:

def make_selection_text(row):
    t = "{prof_name} ---- {dept_acro} {course_num} ----  {course_name} ----  {course_info}"
    return t.format(**row.to_dict())

def make_course_b(row):
    layout = widgets.Layout(width='90%')
    b = widgets.Button(description=make_selection_text(row), button_style='primary', layout=layout)
    
    def handle(event):
        if row['course_id'] in DataStore.course_ids:
            DataStore.remove_course(row['course_id'])
            b.button_style='primary'
        else:
            DataStore.add_course(row['course_id'])
            b.button_style='success'
    b.on_click(handle)
    return b


Run the next cell and select any of the courses which look like yours below

In [15]:
filtered = filter_by_dept_abbrevs(data)
course_buttons =[]
for i, r in filtered.iterrows():
    course_buttons.append(make_course_b(r))

display(widgets.VBox(course_buttons))

# Display potentially stolen material

In [16]:
get_by_course_id(data, DataStore.course_ids)

,Assessments,Assignments,Essays,Homework Help,Lab Reports,Lecture Slides,Lesson Plans,Notes,Test Prep,canonical_course_id,...,prof_id,prof_name,school_name,term,title,total_doc_count,type,url,verified,year
161,NaN,NaN,1,NaN,NaN,NaN,NaN,9,1,1465170,...,296137,Abreu,"California State University, Northridge",3,"California State University, Northridge - HIST...",22,course,/sitemap/schools/272-California-State-Universi...,0,2011
162,1,1,9,5,NaN,NaN,NaN,8,6,296714,...,113967,Thompson,"California State University, Northridge",3,"California State University, Northridge - HIST...",33,course,/sitemap/schools/272-California-State-Universi...,0,2008


In [51]:
from CourseZero.RequestTools import get_content
CHBASE = "https://www.coursehero.com{}"


def get_file_links_from_course_page(course_page_url):
    files = []

    html_content = get_content(CHBASE.format(course_page_url))
    soup = BeautifulSoup(html_content, 'html.parser')

    for j in soup.findAll('a'):
        try:
            dest = j['href']
            fnd = "/file/"
            if dest[: len(fnd)] == fnd:
                files.append(CHBASE.format(dest))
        except Exception as e:
            pass

    files = list(set(files))
    return files

def show_selected(frame):
    selected = get_by_course_id(frame, DataStore.course_ids)
    for i, r in selected.iterrows():
        files = get_file_links_from_course_page(r['url'])
        t = "{prof_name} ---- {dept_acro} {course_num} ----  {course_name} ----  {course_info}"
        print(t.format(**r.to_dict()))
        for f in files:
            print(f)


In [52]:
show_selected(data)

Abreu ---- HIST 101 ----  US History ----  HIST 101 US History
https://www.coursehero.com/file/32127606/313-journaldocx/
https://www.coursehero.com/file/32127640/113-journaldocx/
https://www.coursehero.com/file/17534566/outline-6/
https://www.coursehero.com/file/17534550/Ch3-5-APUSH/
https://www.coursehero.com/file/17534560/ch21-outline/
https://www.coursehero.com/file/27745493/discussiondocx/
https://www.coursehero.com/file/32127637/Untitled-documentdocx/
https://www.coursehero.com/file/28442784/Journals-2docx/
https://www.coursehero.com/file/32127645/Boxer-Reb-Essay-docx/
https://www.coursehero.com/file/17534540/Chapter-1-Study-Guide/
https://www.coursehero.com/file/17534549/Chapter-6-8-Vocabulary-terms-AP-US-HISTORY/
https://www.coursehero.com/file/32127643/Untitled-documentdocx/
https://www.coursehero.com/file/13686428/1-Outline-New-South-Test-Notesdoc/
https://www.coursehero.com/file/32127644/Untitled-documentdocx/
https://www.coursehero.com/file/13686431/5Notes-World-War-One/
htt

In [ ]:
# https://www.coursehero.com/sitemap/schools/272-California-State-University-Northridge/courses/1465170-HIST101/


In [21]:
CHPAGE = "https://www.coursehero.com/sitemap/schools/272-California-State-University-Northridge/courses/1465170-HIST101/"


In [44]:
files = get_file_links_from_course_page(CHPAGE)

In [45]:
files

['https://www.coursehero.com/file/32127606/313-journaldocx/',
 'https://www.coursehero.com/file/32127640/113-journaldocx/',
 'https://www.coursehero.com/file/17534566/outline-6/',
 'https://www.coursehero.com/file/17534550/Ch3-5-APUSH/',
 'https://www.coursehero.com/file/17534560/ch21-outline/',
 'https://www.coursehero.com/file/27745493/discussiondocx/',
 'https://www.coursehero.com/file/32127637/Untitled-documentdocx/',
 'https://www.coursehero.com/file/28442784/Journals-2docx/',
 'https://www.coursehero.com/file/32127645/Boxer-Reb-Essay-docx/',
 'https://www.coursehero.com/file/17534540/Chapter-1-Study-Guide/',
 'https://www.coursehero.com/file/17534549/Chapter-6-8-Vocabulary-terms-AP-US-HISTORY/',
 'https://www.coursehero.com/file/32127643/Untitled-documentdocx/',
 'https://www.coursehero.com/file/13686428/1-Outline-New-South-Test-Notesdoc/',
 'https://www.coursehero.com/file/32127644/Untitled-documentdocx/',
 'https://www.coursehero.com/file/13686431/5Notes-World-War-One/',
 'http

In [ ]:
a = d.copy(deep=True)

In [ ]:
a = a[to_keep]
a

In [ ]:
json_data = get_docs_for_campuses(campus_ids)

In [ ]:
# Query all courses at all csus

abcs = 'abcdefghijklmnopqrstuvwxyz'

data = []

for csu, csuid in campus_ids:
    print(csu)
    for letter in abcs:
        try: 
            url = make_course_search_url(letter, csuid)
            r = requests.get(url, headers=headers)
            data.append(r.json())

        except:
            pass

    
with open(data_json, 'w+') as fpp:
    json.dump(data, fpp)

In [ ]:
dept_fields = ['dept_id', 'dept_acro', 'dept_name']
prof_fields = ['prof_id', 'prof_name']
document_categories = {'Assessments',
 'Assignments',
 'Essays',
 'Homework Help',
 'Lab Reports',
 'Lecture Slides',
 'Lesson Plans',
 'Notes',
 'Reading Lists',
 'Rubrics',
 'Syllabi',
 'Test Prep'}

# load from json

The loaded data will be a list of dictionaries with the format:
    
    {'total': 25,
      'results': [
          {'course_id': '701843',
          ...
          ]
     }

In [ ]:
results = []

for csu in csus:
    r = requests.get(campus_search_url % csu, headers=headers)
    results.append((csu, r.json()))

# drop empty results
# results = [r for r in results if r[0]['id'] != -1 ]

ids = []
for csu, r in results:
    sid = r[0]['id']
    if sid != -1:
        ids.append((csu, sid))

ids    

In [ ]:
with open(data_json, 'r') as f:
    json_data = json.load(f)

In [ ]:
json_data = data

In [ ]:
def get_counts(count_list):
    """Parses out the types of document into a dictionary"""
    d = {}
    for c in count_list:
        d[c['category']] = c['count']
    return d

def make_dict(row):
    """Creates a dictionary for making a dataframe from a record"""
    d = {
        'csu' : row['school_name'],
        'dept' : row['dept_acro'],
        'course_number' : row['course_num'],
        'course_name': row['course_name'],
        'professor': row['prof_name'],
        'total_documents': row['total_doc_count']
    }
    d.update(get_counts(row['doc_counts']))
    return d


In [ ]:
data = pd.DataFrame(data)

In [ ]:
ordered_labels = ['csu', 'dept', 'course_number', 'course_name', 'professor', 'total_documents', 'Assessments', 'Assignments', 'Essays', 'Homework Help', 'Lab Reports',
       'Lecture Slides', 'Lesson Plans', 'Notes', 'Reading Lists', 'Rubrics', 'Syllabi', 'Test Prep']

data = data[ordered_labels]
data

In [ ]:
data[data.csu == 'California Polytechnic State University, Pomona']

In [ ]:
len(data)

In [ ]:
len(set(data['csu'].tolist()))

In [ ]:
data.to_csv(output_file)

In [ ]:
j[:2]

In [ ]:
data.columns

## Save something

In [ ]:
data = pd.DataFrame(data)

In [ ]:
dest = "%s/Desktop/TEMPORARY/dest.csv" % getSystemRoot()
d = data.rename({0: 'csu', 1: 'course_name', 2: 'professor', 3: 'count_of_docs'}, axis=1)
d.to_csv(dest)

In [ ]:
dest

In [ ]:
docs = []
for r in js['results']:
    try:
        docs.append((r['dept_acro'], r['course_name'], r['prof_name'], r['total_doc_count']))
    except:
        pass

In [ ]:
docs

In [ ]:
for r in js['results']:
    print(r)
#     print(r['doc_counts'])

In [ ]:
"doc_counts":[{"category":"Essays","count":"1"},{"category":"Homework Help","count":"2"},{"category":"Notes","count":"5"},{"category":"Test Prep","count":"19"}],"location":{"lat":"35.35432400","lon":"-119.10326200"}

In [ ]:
r.text

In [ ]:
csus = [
        'California State University Bakersfield',
        'California State University Channel islands', 
        'California State University Chico', 
        'California State University Dominguez hills', 
        'California State University East bay',
        'California State University Fresno', 
        'California State University Fullerton', 
        'California State University Long Beach',    
        'California State University Los Angeles',
        'California State University Northridge', 
        'California State University Monterey Bay',
        'California Polytechnic State University Pomona', 
        'California Polytechnic State University San Luis Obispo', 
        'California State University San Bernardino',
        'California State University San Marcos',
        'California State University Stanislaus',
        'California Maritime Academy',
        'Humboldt State University',     
        'Sacramento State University',
        'San Diego State University', 
        'San Francisco State University', 
        'San Jose state University', 
        'Sonoma State University',
       ]

In [ ]:
user_agent = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36"
headers = {'User-Agent': user_agent}
